In [1]:
import pandas as pd

## Take a look at the column headers and first row to decide data types

In [2]:
df_one = pd.read_csv('MetObjects_NoMultiline.csv',sep=',',encoding='utf-8',nrows=1,dtype='str')
df_one.loc[0,:]

Object Number                                                     1979.486.1
Is Highlight                                                           False
Is Public Domain                                                       False
Is Timeline Work                                                       False
Object ID                                                                  1
Department                                                 The American Wing
AccessionYear                                                           1979
Object Name                                                             Coin
Title                                           One-dollar Liberty Head Coin
Culture                                                                  NaN
Period                                                                   NaN
Dynasty                                                                  NaN
Reign                                                                    NaN

### Set column data types

In [3]:
dtypes = {
    "Object Number":"str",
    "Is Highlight":"bool",
    "Is Public Domain":"bool",
    "Is Timeline Work":"bool",
    "Object ID":"int",
    "Department":"category",
    "AccessionYear":"str",
    "Object Name":"category",
    "Title":"str",
    "Culture":"category",
    "Period":"category",
    "Dynasty":"category",
    "Reign":"category",
    "Portfolio":"str",
    "Artist Role":"category",
    "Artist Prefix":"category",
    "Artist Display Name":"str",
    "Artist Display Bio":"str",
    "Artist Suffix":"category",
    "Artist Alpha Sort":"str",
    "Artist Nationality":"category",
    "Artist Begin Date":"str",
    "Artist End Date":"str",
    "Artist Gender":"category",
    "Artist ULAN URL":"str",
    "Artist Wikidata URL":"str",
    "Object Date":"str",
    "Object Begin Date":"str",
    "Object End Date":"str",
    "Medium":"category",
    "Dimensions":"str",
    "Credit Line":"str",
    "Geography Type":"category",
    "City":"category",
    "State":"category",
    "County":"category",
    "Country":"category",
    "Region":"category",
    "Subregion":"category",
    "Locale":"category",
    "Locus":"category",
    "Excavation":"category",
    "River":"category",
    "Classification":"category",
    "Rights and Reproduction":"category",
    "Link Resource":"str",
    "Object Wikidata URL":"str",
    "Metadata Date":"str",
    "Repository":"category",
    "Tags":"category",
    "Tags AAT URL":"category"
}

In [5]:
df = pd.read_csv(
    'MetObjects_NoMultiline.csv',
    sep=',',
    encoding='utf-8',
    dtype=dtypes,
    true_values=['true','True','TRUE'],
    false_values=['false','False','FALSE']
)

### Just forming some convenience lists for exploring value types

In [7]:
cat_columns = [k for k,v in dtypes.items() if v=='category']
bool_columns = [k for k,v in dtypes.items() if v=='bool']
int_columns = [k for k,v in dtypes.items() if v=='int']
str_columns = [k for k,v in dtypes.items() if v=='str']

## Explore the range and dropoff in unique values for each type of column

In [8]:
for col in bool_columns:
    print(col)
    print(df[col].value_counts())
    print("\n")

Is Highlight
False    472467
True       2008
Name: Is Highlight, dtype: int64


Is Public Domain
False    466505
True       7970
Name: Is Public Domain, dtype: int64


Is Timeline Work
False    239913
True     234562
Name: Is Timeline Work, dtype: int64




## Load in "on view" data that came from an API call

[https://collectionapi.metmuseum.org/public/collection/v1/search?isOnView=True&q=\*]()

In [14]:
df_onview = pd.read_json('OnViewTrue_Qstar.json')
df_onview.head()

,total,objectIDs
0,36587,254738
1,36587,254747
2,36587,254735
3,36587,254741
4,36587,254733


In [15]:
df_onview['on_view_api'] = True
df_onview.drop(columns=['total'], inplace=True)
df_onview.head()

,objectIDs,on_view_api
0,254738,True
1,254747,True
2,254735,True
3,254741,True
4,254733,True


### Just checking data type of IDs since it should match with original table for JOIN

In [16]:
df_onview.dtypes

objectIDs      int64
on_view_api     bool
dtype: object

## Merge OnDisplay with original table

### First, make a quick check that there aren't any extra IDs

Want to make sure there aren't any IDs in the API call JSON data that weren't in the original table

In [19]:
df_onview.merge(df['Object ID'],
                left_on="objectIDs",
                right_on="Object ID",
                how='left')['Object ID'].isna().sum()

0

In [21]:
df2 = df.merge(df_onview,
         left_on="Object ID",
         right_on="objectIDs",
         how='left'
).drop(columns=['objectIDs'])

### Boolean and integer columns evidently can't have NaN values

So, on join "on_view_api" gets cast as a string/object. 

**This may indicate that all of the True/False columns should be categorical or Int64 rather than boolean!!**

In [24]:
df2['on_view_api']

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
474470     NaN
474471     NaN
474472     NaN
474473     NaN
474474    True
Name: on_view_api, Length: 474475, dtype: object

### Not getting any NaN values in other columns on the Left, so I think all of the IDs in the OnDisplay are valid

In [22]:
bool_columns2 = bool_columns + ['on_view_api']

for col in bool_columns2:
    print(col)
    print(df2[col].value_counts(dropna=False))
    print("\n")

Is Highlight
False    472467
True       2008
Name: Is Highlight, dtype: int64


Is Public Domain
False    466505
True       7970
Name: Is Public Domain, dtype: int64


Is Timeline Work
False    239913
True     234562
Name: Is Timeline Work, dtype: int64


OnDisplay
NaN     437888
True     36587
Name: OnDisplay, dtype: int64




## Save file including new column

In [28]:
df2.to_csv("MetObjects_OnDisplay.tsv",
           sep='\t',
           index=False,
           encoding='utf-8'
          )